### Player group (team) classification

In [ ]:
from collections import Counter

import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
import sklearn.linear_model as lm
from lightgbm import LGBMClassifier
import warnings; warnings.filterwarnings('ignore')
import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
X_df = pd.read_pickle("X_df")
y_df = pd.read_pickle("y_df")

X_12 = X_df[y_df.isin([1,2])]
y_12 = list(y_df[y_df.isin([1,2])])
Counter(y_df), Counter(y_12)

### Training classifier on few frames

In [ ]:
def fit_classifier(model, row_from, row_to, data_X, data_y):
    model = model.fit(data_X[row_from: row_to], data_y[row_from: row_to])
    preds = model.predict_proba(data_X[row_to:])
    max_preds = preds.argmax(axis=1)
    if 0 not in set(data_y):
        max_preds += 1
    print('Confusion matrix:\n', confusion_matrix(data_y[row_to:], max_preds))
    plt.hist(preds.max(axis=1))

In [ ]:
fit_classifier(lm.LogisticRegression(C=300.0), 0, 300, X_12, y_12)

In [ ]:
fit_classifier(lm.LogisticRegression(C=300.0), 0, 600, X_12, y_12)

In [ ]:
fit_classifier(lm.LogisticRegression(C=300.0), 0, 600, X_df, y_df)

In [ ]:
fit_classifier(LGBMClassifier(), 0, 300, X_12, y_12)

In [ ]:
fit_classifier(LGBMClassifier(), 0, 600, X_df, y_df)

In [ ]:
def partial_fit_classifier(model, max_number_of_frames_train, X, y, classes):
    for i in range(0, 20*max_number_of_frames_train, 20):
        model = model.partial_fit(X[i:i+20], y[i:i+20], classes=classes)
        preds = model.predict(X[i+20:])
        number_of_frames_test = (len(X) - (i + 20) ) // 20
        print(f'Confusion matrix. {number_of_frames_test} test frames (fitted on first {(i+20)//2} frames):')
        print(confusion_matrix(preds, y[i+20:]))

In [ ]:
sgd = lm.SGDClassifier(loss='hinge', penalty='l2')
partial_fit_classifier(sgd, 50, X_12, y_12, [1, 2])

In [ ]:
partial_fit_classifier(BernoulliNB(alpha=0.1, binarize=0.0), 50, X_12, y_12, [1, 2])
    

In [ ]:
partial_fit_classifier(MultinomialNB(), 50, X_12, y_12, [1, 2])

In [ ]:
partial_fit_classifier(GaussianNB(), 50, X_12, y_12, [1, 2])

In [ ]:
X_012 = X_df[y_df.isin([0, 1, 2])]
y_012 = list(y_df[y_df.isin([0, 1, 2])])
partial_fit_classifier(BernoulliNB(alpha=0.1, binarize=0.0), 50, X_012, y_012, [0, 1, 2])

In [ ]:
partial_fit_classifier(MultinomialNB(), 50, X_012, y_012, [0, 1, 2])